# Лабораторная работа №3
## Выполнил студент группы БФИ-2001 Никитин Г.А

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнение лабораторной

In [8]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as ti
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict


### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [27]:
start_time = ti.now()
s = "kpf"
t = "gfsahgsafykpfjssavv dhya98usa"

def prefix(s):
    v = [0]*len(s)
    for i in range(1,len(s)):
        k = v[i-1]
        while k > 0 and s[k] > s[i]:
            k = v[k-1]
        if s[k] == s[i]:
            k = k + 1
        v[i] = k
    return v

def kmp(s,t):
    index = -1
    f = prefix(s)
    k = 0
    for i in range(len(t)):
        while k > 0 and s[k] > t[i]:
            k = f[k-1]
        if s[k] == t[i]:
            k = k + 1
        if k == len(s):
            index = i - len(s) + 1
            break
    return index
print(f"{(ti.now() - start_time).microseconds} ms")
kmp(s,t)

478 ms


10

#### Упрощенный алгоритм Бойера-Мура

In [28]:
start_time = ti.now()
s = "kpf"
t = "gfsahgsafykpfjssavv dhya98usa"

class last_occurrence(object):

    def __init__(self, pattern, alphabet):
        self.occurrences = dict()
        for letter in alphabet:
            self.occurrences[letter] = pattern.rfind(letter)

    def __call__(self, letter):
        return self.occurrences[letter]

def bms(text, pattern):
    alphabet = set(text)
    last = last_occurrence(pattern, alphabet)
    m = len(pattern)
    n = len(text)
    i = m - 1  
    j = m - 1  
    while i < n:
        if text[i] == pattern[j]:
            if j == 0:
                return i
            else:
                i -= 1
                j -= 1
        else:
            l = last(text[i])
            i = i + m - min(j, 1+l)
            j = m - 1 
    return -1
print(f"{(ti.now() - start_time).microseconds} ms")
bms(t,s)


354 ms


10

### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента. 

In [22]:
def check_15_valid(table):
    n = 0
    for i in range(1, 16):
        inx = table.index(i)
        row = (inx // 4) + 1
        count = 0
        for j in range(inx + 1, 16):
            v = table[j]
            if v > 0 and v < i: count += 1
        n += count + row 
    return n % 2 == 0

def cell_pos(index):
    return (index % 4, index // 4)

def cell_index(pos):
    return pos[0] * 4 + pos[1]

class AStarNode:
    def __init__(self, parent, step, distance, table, moved_num):
        self.parent = parent
        self.step = step
        self.distance = distance
        self.table = table
        self.space_index = table.index(0)
        self.space_position = cell_pos(self.space_index)
        self.moved_num = moved_num

    def __eq__(self, other):
        if isinstance(other, AStarNode):
            return self.table == other.table
        return False

    def __ne__(self, other):
        return not self.__eq__(other)
    
    @property
    def price(self):
        return self.step + self.distance * 10

    def get_neighbors(self):
        space_position = self.space_position
        result = []
        if space_position[0] > 0:
            result.append(self.get_neighbor((-1, 0)))
        if space_position[0] < 3:
            result.append(self.get_neighbor((1, 0)))
        if space_position[1] > 0:
            result.append(self.get_neighbor((0, -1)))
        if space_position[1] < 3:
            result.append(self.get_neighbor((0, 1)))
        return result
    
    def get_neighbor(self, move):
        new_table = self.table.copy()
        new_cell_index = cell_index((self.space_position[0] + move[0], self.space_position[1] + move[1]))
        moved_num = new_table[new_cell_index]
        new_table[self.space_index], new_table[new_cell_index] = moved_num, new_table[self.space_index]
        return AStarNode(self, self.step + 1, calc_distance(new_table), new_table, moved_num)

def calc_distance(table):
    result = 0
    for i in range(0, 16):
        if table[i] != i + 1 and table[i] > 0:
            result += 1
    return result

def resolve_15(table):
    if not check_15_valid(table): return []
    
    open = [AStarNode(None, 0, calc_distance(table), table, 0)]
    close = []

    def get_min_open():
        min_node = None
        min_price = 99999
        for n in open:
            if n.price < min_price:
                min_price = n.price
                min_node = n
        return min_node

    i = 0

    while len(open) > 0:
        current = get_min_open()
        #print(f"--- {i} ---")
        #print((current.step, current.distance))
        #print(current.table)

        i += 1

        if calc_distance(current.table) == 0:
            result = []
            while current is not None and current.moved_num > 0:
                result.append(current.moved_num)
                current = current.parent
            result.reverse()
            return result
        
        open.remove(current)
        close.append(current)
        neighbors = current.get_neighbors()
        for neighbor in neighbors:
            if neighbor in close: continue
            step = current.step + 1
            isBetter = False
            if neighbor not in open:
                neighbor.distance = calc_distance(neighbor.table)
                open.append(neighbor)
                isBetter = True
            else:
                isBetter = step < neighbor.step
            if isBetter:
                neighbor.parent = current
                neighbor.step = step

table = [7, 1, 3, 14, 6, 5, 2, 8, 12, 10, 11, 9, 13, 4, 15, 0]
resolve_15(table)

[]

### Вывод